# 1. Set up parameters for ASL

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from obspy import read_inventory
from importlib import reload
from flovopy.asl.wrappers import run_single_event, find_event_files, run_all_events
from flovopy.core.mvo import dome_location, REGION_DEFAULT
from flovopy.processing.sam import VSAM, DSAM 
from flovopy.asl.config import ASLConfig, tweak_config
# -------------------------- Config --------------------------
# directories
HOME = Path.home()
PROJECTDIR      = HOME / "Dropbox" / "BRIEFCASE" / "SSADenver"
LOCALPROJECTDIR = HOME / "work" / "PROJECTS" / "SSADenver_local"
OUTPUT_DIR      = LOCALPROJECTDIR / "ASL_RESULTS"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
INPUT_DIR       = PROJECTDIR / "ASL_inputs" / "biggest_pdc_events"
GLOBAL_CACHE    = PROJECTDIR / "asl_global_cache"
METADATA_DIR    = PROJECTDIR / "metadata" 
STATION_CORRECTIONS_DIR = PROJECTDIR / "station_correction_analysis"

# master files
INVENTORY_XML   = METADATA_DIR / "MV_Seismic_and_GPS_stations.xml"
DEM_DEFAULT     = METADATA_DIR / "MONTSERRAT_DEM_WGS84_MASTER.tif"
GRIDFILE_DEFAULT= METADATA_DIR / "MASTER_GRID_MONTSERRAT.pkl"

# parameters for envelopes and cross-correlation
SMOOTH_SECONDS  = 1.0
MAX_LAG_SECONDS = 8.0
MIN_XCORR       = 0.5

# other parameters
DIST_MODE = "3d" # or 2d. will essentially squash Montserrat topography and stations onto a sea-level plane, ignored elevation data, e.g. for computing distances

# Inventory of Montserrat stations
INV     = read_inventory(INVENTORY_XML)
print(f"[INV] Networks: {len(INV)}  Stations: {sum(len(n) for n in INV)}  Channels: {sum(len(sta) for net in INV for sta in net)}")

# Montserrat station corrections estimated from regionals
station_corrections_csv = STATION_CORRECTIONS_DIR / "station_gains_intervals.csv"
annual_station_corrections_csv = STATION_CORRECTIONS_DIR / "station_gains_intervals_by_year.csv"
station_corrections_df = pd.read_csv(station_corrections_csv)
annual_station_corrections_df = pd.read_csv(annual_station_corrections_csv)

# Montserrat pre-defined Grid (from 02 tutorial)
from flovopy.asl.grid import Grid
gridobj = Grid.load(GRIDFILE_DEFAULT)
print(gridobj)
landgridobj = Grid.load(GLOBAL_CACHE / "land" / "Grid_9c2fd59b.pkl")

# Montserrat constants
from flovopy.core.mvo import dome_location, REGION_DEFAULT
print("Dome (assumed source) =", dome_location)

# events and wrappers
event_files = list(find_event_files(INPUT_DIR))
eventcsvfile = Path(OUTPUT_DIR) / "mseed_files.csv"
if not eventcsvfile.is_file():
    rows = [{"num": num, "f": str(f)} for num, f in enumerate(event_files)]
    df = pd.DataFrame(rows)
    df.to_csv(eventcsvfile, index=False)
best_file_nums  = [35, 36, 40, 52, 82, 83, 84, 116, 310, 338]
best_event_files = [event_files[i] for i in best_file_nums]
print(f'Best miniseed files are: {best_event_files}')
REFINE_SECTOR = False   # enable triangular dome-to-sea refinement

# Parameters to pass for making pygmt topo maps
topo_kw = {
    "inv": INV,
    "add_labels": True,
    "cmap": "gray",
    "region": REGION_DEFAULT,
    "dem_tif": DEM_DEFAULT,  # basemap shading from your GeoTIFF - but does not actually seem to use this unless topo_color=True and cmap=None
    "frame": True,
    "dome_location": dome_location,
}

# Build a baseline configuration
This is inherited by various downstream functions
This describes the physical parameters, the station metadata, the grid, the misfit algorithm, etc.

In [ ]:
DEBUG=False
baseline_cfg = ASLConfig(
    inventory=INV,
    output_base=OUTPUT_DIR,
    gridobj=gridobj,
    global_cache=GLOBAL_CACHE,
    station_correction_dataframe=station_corrections_df,
    wave_kind="surface",
    speed=1.5,
    Q=23, 
    peakf=2.0,
    dist_mode="3d", 
    misfit_engine="r2",
    window_seconds=5.0,
    min_stations=5,
    sam_class=VSAM, 
    sam_metric="mean",
    debug=DEBUG,
)
baseline_cfg.build()

# Run one event with this baseline configuration

In [ ]:
DEBUG=True
result = run_single_event(
    mseed_file=event_files[0],
    cfg=baseline_cfg,
    refine_sector=False,
    station_gains_df=None,
    switch_event_ctag = True,
    topo_kw=topo_kw,
    mseed_units='m/s', # default units for miniseed files being used - probably "Counts" or "m/s"        
    reduce_time=True,
    debug=DEBUG,
)

# Run events (one event=one miniseed file) with this baseline configuration

In [ ]:
summaries = []
REFINE_SECTOR=False
for i, ev in zip(best_file_nums, best_event_files):
    print(f"[{i}/{len(event_files)}] {ev}")
    result = run_single_event(
        mseed_file=str(ev),
        cfg=baseline_cfg,
        refine_sector=REFINE_SECTOR,
        station_gains_df=None,
        switch_event_ctag = True,
        topo_kw=topo_kw,
        mseed_units='m/s', # default units for miniseed files being used - probably "Counts" or "m/s"        
        reduce_time=True,
        debug=DEBUG,
    )
    summaries.append(result)

# Summarize
df = pd.DataFrame(summaries)
display(df)

summary_csv = Path(OUTPUT_DIR) / f"{baseline_cfg.tag()}__summary.csv"
df.to_csv(summary_csv, index=False)
print(f"Summary saved to: {summary_csv}")

if not df.empty:
    n_ok = int((~df.get("error").notna()).sum()) if "error" in df.columns else len(df)
    print(f"Success: {n_ok}/{len(df)}")

# Easily create new configurations that change 1 or 2 parameters
Here we create 18 new configurations, each is an entry in the changes dictionary.

In [ ]:
variants = tweak_config(
    baseline_cfg,
    changes=[
        {"Q": 10},                                      # decrease Q from 23 to 10
        {"Q": 100},                                     # increase Q from 23 to 100
        {"speed": 0.5},                                 # decrease wave speed from 1.5 km/s to 0.5 km/s
        {"speed": 2.5},                                 # increase wave speed from 1.5 km/s to 2.5 km/s
        {"peakf": 8.0},                                 # increae peakf from 2.0 Hz to 8.0 Hz
        {"dist_mode": "2d"},                            # change from 3D to 2D: ignore terrain & ignore station elevations
        {"station_correction_dataframe": None},         # turn off station corrections
        {"gridobj":landgridobj},                        # try a grid that allows whole Southern end of island, not just dome & ravines
        {"misfit_engine": "l2"},                        # change the misfit function from r2 to l2
        {"misfit_engine": "lin"},                       # change the misfit function from r2 to lin
        {"window_seconds": 1.0},                        # decrease the moving window length from 5-s to 1-s
        {"sam_class": DSAM},                            # switch from VELOCITY Seismic Amplitude Measurement (VSAM) to DISPLACEMENT Seismic Amplitude Measurement (DSAM)
        {"sam_metric": "median"},                       # switch from MEAN of each 5-s moving time window, to MEDIAN
        {"sam_metric": "rms"},                          # switch from MEAN of each 5-s moving time window, to RMS
        {"sam_metric": "max"},                          # switch from MEAN of each 5-s moving time window, to MAX
        {"sam_metric": "LP"},                           # switch from MEAN in 0.5-18.0 Hz band to mean in LP band (0.5-4.0 Hz)
        {"sam_metric": "VT"},                           # switch from MEAN in 0.5-18.0 Hz band to mean in VT band (4.0-18.0 Hz)
        {"wave_kind": "body", "speed": 2.5},            # change multiple params: surface->body waves, wave speed 1.5->3.0 km/s - THIS IS A REFERENCE TO COMPARE SURFACE WAVES AND BODY WAVES
    ],
)
print(variants)

# Run pairs: variant vs baseline

In [ ]:
from flovopy.asl.compare_runs import compare_runs

scored, summary, win_counts = compare_runs(
    baseline_cfg,
    events=best_event_files,
    variants=variants,
    run_single_event=run_single_event,
    refine_sector=False,
    topo_kw=topo_kw,
    run_if_missing_baseline=True,
    run_if_missing_variants=True,
    # the following numbers should add up to 1.0. if truly just want to see difference RELATIVE to baseline, set w_sep=1.0, and others to 0.0. for ABSOLUTE quality check, set w_sep to 0.0
    w_sep=0.5, # set this high to penalize large location difference from the baseline config
    w_misfit=0.2, # set this high to punish high misfits
    w_azgap=0.1, # punish larger azimuthal gaps
    w_conn=0.1, # reward more connectedness
    w_rough=0.1, # reward less roughness / more straightness
)

# Inspect/save
if scored is not None:
    display(summary)
    display(win_counts)
    summary.to_csv(OUTPUT_DIR / "pairwise_summary_surface.csv", index=False)

In [ ]:
scored, summary, win_counts = compare_runs(
    baseline_cfg,
    events=best_event_files,
    variants=variants,
    run_single_event=run_single_event,
)


## baseline-free absolute scoring


In [ ]:
from flovopy.asl.compare_runs import add_baseline_free_scores, summarize_absolute_runs, per_event_winner_abs, crawl_intrinsic_runs

# Option B: OR just crawl everything that already exists under OUTPUT_DIR
abs_tbl = crawl_intrinsic_runs(OUTPUT_DIR)
weights = {
    "mean_misfit":     1.0,   # lower better
    "mean_azgap":      0.1,   # lower better
    "roughness_ratio": 0.1,   # lower better
    "connectedness":  -0.3,   # higher better (negative weight)
    "valid_frac":     -0.2,   # higher better (negative weight)
}
abs_scored = add_baseline_free_scores(abs_tbl, weights=weights)

abs_summary = summarize_absolute_runs(abs_scored)
winners_abs, win_counts_abs = per_event_winner_abs(abs_scored)

display(abs_summary)
display(win_counts_abs)

abs_summary.to_csv(OUTPUT_DIR / "absolute_summary.csv", index=False)

# Now let's try to run a suite of body wave configurations


In [ ]:
# Create new baseline configuration - this time for body waves
baseline_cfg = ASLConfig(
    inventory=INV,
    output_base=OUTPUT_DIR,
    gridobj=gridobj,
    global_cache=GLOBAL_CACHE,
    station_correction_dataframe=station_corrections_df,
    wave_kind="body",
    speed=2.5,
    Q=23, 
    peakf=2.0,
    dist_mode="3d", 
    misfit_engine="r2",
    window_seconds=5.0,
    min_stations=5,
    sam_class=VSAM, 
    sam_metric="mean",
    debug=DEBUG,
)
baseline_cfg.build()

# Easily create new configurations - 18 new ones
variants = tweak_config(
    baseline_cfg,
    changes=[
        {"Q": 10},                                      # decrease Q from 23 to 10
        {"Q": 100},                                     # increase Q from 23 to 100
        {"speed": 1.5},                                 # decrease wave speed from 2.5 km/s to 1.5 km/s
        {"speed": 4.0},                                 # increase wave speed from 2.5 km/s to 4.0 km/s
        {"peakf": 8.0},                                 # increae peakf from 2.0 Hz to 8.0 Hz
        {"dist_mode": "2d"},                            # change from 3D to 2D: ignore terrain & ignore station elevations
        {"station_correction_dataframe": None},         # turn off station corrections
        {"gridobj":landgridobj},                        # try a grid that allows whole Southern end of island, not just dome & ravines
        {"misfit_engine": "l2"},                        # change the misfit function from r2 to l2
        {"misfit_engine": "lin"},                       # change the misfit function from r2 to lin
        {"window_seconds": 1.0},                        # decrease the moving window length from 5-s to 1-s
        {"sam_class": DSAM},                            # switch from VELOCITY Seismic Amplitude Measurement (VSAM) to DISPLACEMENT Seismic Amplitude Measurement (DSAM)
        {"sam_metric": "median"},                       # switch from MEAN of each 5-s moving time window, to MEDIAN
        {"sam_metric": "rms"},                          # switch from MEAN of each 5-s moving time window, to RMS
        {"sam_metric": "max"},                          # switch from MEAN of each 5-s moving time window, to MAX
        {"sam_metric": "LP"},                           # switch from MEAN in 0.5-18.0 Hz band to mean in LP band (0.5-4.0 Hz)
        {"sam_metric": "VT"},                           # switch from MEAN in 0.5-18.0 Hz band to mean in VT band (4.0-18.0 Hz)
        {"wave_kind": "surface", "speed": 1.0},         # change multiple params: surface->body waves, wave speed 1.5->3.0 km/s - THIS IS A REFERENCE TO COMPARE SURFACE WAVES AND BODY WAVES
    ],
)

# Run pairs - variant versus baseline
scored, summary, win_counts = compare_runs(
    baseline_cfg,
    events=best_event_files,
    variants=variants,
    run_single_event=run_single_event,
    refine_sector=False,
    topo_kw=topo_kw,
    run_if_missing_baseline=True,
    run_if_missing_variants=True,
    # the following numbers should add up to 1.0. if truly just want to see difference RELATIVE to baseline, set w_sep=1.0, and others to 0.0. for ABSOLUTE quality check, set w_sep to 0.0
    w_sep=0.5, # set this high to penalize large location difference from the baseline config
    w_misfit=0.2, # set this high to punish high misfits
    w_azgap=0.1, # punish larger azimuthal gaps
    w_conn=0.1, # reward more connectedness
    w_rough=0.1, # reward less roughness / more straightness
)

if scored is not None:
    display(summary)
    display(win_counts)
    summary.to_csv(OUTPUT_DIR / "pairwise_summary_body.csv", index=False)


# baseline-free absolute scoring
abs_tbl = crawl_intrinsic_runs(OUTPUT_DIR)
weights = {
    "mean_misfit":     1.0,   # lower better
    "mean_azgap":      0.1,   # lower better
    "roughness_ratio": 0.1,   # lower better
    "connectedness":  -0.3,   # higher better (negative weight)
    "valid_frac":     -0.2,   # higher better (negative weight)
}
abs_scored = add_baseline_free_scores(abs_tbl, weights=weights)

abs_summary = summarize_absolute_runs(abs_scored)
winners_abs, win_counts_abs = per_event_winner_abs(abs_scored)

display(abs_summary)
display(win_counts_abs)

abs_summary.to_csv(OUTPUT_DIR / "absolute_summary.csv", index=False)

In [2]:
print(landgridobj)

NameError: name 'landgridobj' is not defined